In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

In [ ]:
AVG_LEN_THRES = 8

#Process Kaggle Questions (Science and Miscellaneous) Dataset

In [ ]:
miscel = pd.read_csv('miscel.csv')
sci = pd.read_csv('sci.csv')

Remove Noise

In [ ]:
condition = [len(q.split(" ")) >= 3 for q in miscel['prompt']]
miscel = miscel[condition]

condition = [len(q.split(" ")) >= 3 for q in sci['prompt']]
sci = sci[condition]

In [ ]:
def clean_words(row):
  if "wikipedia excerpt" in str(row['A']).lower():
    row['A'] = row['A'].replace("Wikipedia excerpt", "context")
    row['A'] = row['A'].replace("wikipedia excerpt", "context")
  if "wikipedia excerpt" in str(row['B']).lower():
    row['B'] = row['B'].replace("Wikipedia excerpt", "context")
    row['B'] = row['B'].replace("wikipedia excerpt", "context")
  if "wikipedia excerpt" in str(row['C']).lower():
    row['C'] = row['C'].replace("Wikipedia excerpt", "context")
    row['C'] = row['C'].replace("wikipedia excerpt", "context")
  if "wikipedia excerpt" in str(row['D']).lower():
    row['D'] = row['D'].replace("Wikipedia excerpt", "context")
    row['D'] = row['D'].replace("wikipedia excerpt", "context")
  if "wikipedia excerpt" in str(row['E']).lower():
    row['E'] = row['E'].replace("Wikipedia excerpt", "context")
    row['E'] = row['E'].replace("wikipedia excerpt", "context")
  if "wikipedia excerpt" in str(row['prompt']).lower():
    row['prompt'] = row['prompt'].replace("Wikipedia excerpt", "context")
    row['prompt'] = row['prompt'].replace("wikipedia excerpt", "context")

  if "excerpt" in str(row['A']).lower():
    row['A'] = row['A'].replace("excerpt", "context")
  if "excerpt" in str(row['B']).lower():
    row['B'] = row['B'].replace("excerpt", "context")
  if "excerpt" in str(row['C']).lower():
    row['C'] = row['C'].replace("excerpt", "context")
  if "excerpt" in str(row['D']).lower():
    row['D'] = row['D'].replace("excerpt", "context")
  if "excerpt" in str(row['E']).lower():
    row['E'] = row['E'].replace("excerpt", "context")
  if "excerpt" in str(row['prompt']).lower():
    row['prompt'] = row['prompt'].replace("excerpt", "context")

  return row

In [ ]:
miscel = miscel.apply(clean_words, axis=1)
sci = sci.apply(clean_words, axis=1)

Extract Useful Cells

In [ ]:
final_kaggle_dataset = pd.DataFrame(columns=['context', 'question', 'answer', 'dis1', 'dis2', 'dis3', 'source_dataset']) # final kaggle dataset

In [ ]:
def count_words(string):
  return len(str(string).split(' '))

In [ ]:
for r in miscel.iterrows():
  answer_id = r[1]['answer']
  answer = r[1][answer_id]

  dis_idc = [d for d in ['A', 'B', 'C', 'D', 'E'] if d != answer_id]
  distractors = [r[1][dis_id] for dis_id in dis_idc]
  distractors = sorted(distractors, key=count_words, reverse=True)[:3] # get 3 distractors only

  dis_lens = [len(str(d).split(' ')) for d in distractors]
  avg_dis_len = pd.Series(dis_lens).mean()

  if avg_dis_len >= AVG_LEN_THRES:
    datapoint = {"context": [r[1]['wikipedia_excerpt']], "question": [r[1]['prompt']], "answer": [answer], "dis1": [distractors[0]], "dis2": [distractors[1]], "dis3": [distractors[2]], 'source_dataset': ['kaggle']}
    final_kaggle_dataset = pd.concat([final_kaggle_dataset, pd.DataFrame(datapoint)], axis=0, ignore_index=True)

for r in sci.iterrows():
  answer_id = r[1]['answer']
  answer = r[1][answer_id]

  dis_idc = [d for d in ['A', 'B', 'C', 'D', 'E'] if d != answer_id]
  distractors = [r[1][dis_id] for dis_id in dis_idc]
  distractors = sorted(distractors, key=count_words, reverse=True)[:3] # get 3 distractors only

  dis_lens = [len(str(d).split(' ')) for d in distractors]
  avg_dis_len = pd.Series(dis_lens).mean()

  if avg_dis_len >= AVG_LEN_THRES:
    datapoint = {"context": [r[1]['wikipedia_excerpt']], "question": [r[1]['prompt']], "answer": [answer], "dis1": [distractors[0]], "dis2": [distractors[1]], "dis3": [distractors[2]]}
    final_kaggle_dataset = pd.concat([final_kaggle_dataset, pd.DataFrame(datapoint)], axis=0, ignore_index=True)

In [ ]:
# remove error
final_kaggle_dataset = final_kaggle_dataset[final_kaggle_dataset['question'] != "What are some models that attempt to account for all observations without invoking supplemental non-baryonic matter?"]
final_kaggle_dataset = final_kaggle_dataset.reset_index(drop=True)

In [ ]:
# additional cleansing

filtered_ids = []
for r in final_kaggle_dataset.iterrows():
  if not ("||" in r[1]['context']):
    filtered_ids.append(r[0])

final_kaggle_dataset = final_kaggle_dataset.iloc[filtered_ids, :]
final_kaggle_dataset = final_kaggle_dataset.reset_index(drop=True)

In [ ]:
print(miscel.shape)
print(sci.shape)
print(final_kaggle_dataset.shape)

(5998, 8)
(6019, 8)
(8228, 7)


#Process RACE Dataset

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

race = load_dataset("ehovy/race", "all")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
race_dataset = pd.concat([pd.DataFrame(race['train']), pd.DataFrame(race['validation']), pd.DataFrame(race['test'])], axis=0)

Remove Noise

In [ ]:
race_dataset = race_dataset.dropna()
race_dataset = race_dataset.drop(columns='example_id', axis=1)

In [ ]:
condition = [len(q.split(" ")) >= 3 for q in race_dataset['question']]
race_dataset = race_dataset[condition]

Extract Useful Data

In [ ]:
final_race_dataset = pd.DataFrame(columns=['context', 'question', 'answer', 'dis1', 'dis2', 'dis3', 'source_dataset']) # final kaggle dataset

In [ ]:
def remove_trailing_blank(text):
  """Remove trailing blank (if found), and replace it with question mark ('?')
  """
  words = [char for char in text]

  b_pos = text.rfind('_')
  if b_pos == -1:
    return text

  if b_pos != 0:
    if words[b_pos - 1] == ' ':
      b_pos -=1

  words = words[:b_pos]
  words.append('?')

  return ''.join(words)

In [ ]:
for r in race_dataset.iterrows():
  options = r[1]['options']

  option_ids = ['A', 'B', 'C', 'D']
  answer_id = option_ids.index(r[1]['answer'])

  if r[1]['question'].count('_') >= 2:
    continue
  question = remove_trailing_blank(r[1]['question'])

  # get answer and distractors
  answer = options[answer_id]
  distractors = [options[ind] for ind in range(len(options)) if ind != answer_id]
  assert len(distractors) == 3

  dis_lens = [len(str(d).split(' ')) for d in distractors]
  avg_dis_len = pd.Series(dis_lens).mean()

  if avg_dis_len >= AVG_LEN_THRES:
    datapoint = {"context": [r[1]['article']], "question": [question], "answer": [answer], "dis1": [distractors[0]], "dis2": [distractors[1]], "dis3": [distractors[2]], 'source_dataset': ['race']}
    final_race_dataset = pd.concat([final_race_dataset, pd.DataFrame(datapoint)], axis=0, ignore_index=True)

In [ ]:
# additional cleansing: remove questions requiring summary-related understanding, not poking at specific information
sum_ques_keywords = ['main idea', 'passage', 'article', 'which is true', 'is true', 'is not true', 'is false', 'is correct', 'is not correct', 'statement', 'author', 'writer', 'conclude', 'conclusion', 'infer', 'inferred', 'tell us'] # key words found in questions requiring summarization

filtered_ids = []
for r in final_race_dataset.iterrows():
  if not any([key in r[1]['question'].lower() for key in sum_ques_keywords]):
    filtered_ids.append(r[0])

final_race_dataset = final_race_dataset.iloc[filtered_ids]
final_race_dataset = final_race_dataset.reset_index(drop=True)

In [ ]:
temp_ids = []
for r in final_race_dataset.iterrows():
  if '_' in r[1]['question']:
    temp_ids.append(r[0])

Process CosmosQA

In [ ]:
from datasets import load_dataset

cosmosqa = load_dataset("cosmos_qa")

In [ ]:
cosmosqa_dataset = pd.concat([pd.DataFrame(cosmosqa['train']), pd.DataFrame(cosmosqa['validation']), pd.DataFrame(cosmosqa['test'])], axis=0)

In [ ]:
cosmosqa_dataset.drop(labels=['id'], axis=1, inplace=True)
cosmosqa_dataset = cosmosqa_dataset.dropna()

In [ ]:
cosmosqa_dataset.head(1)

,context,question,answer0,answer1,answer2,answer3,label
0,"Good Old War and person L : I saw both of these bands Wednesday night , and they both blew me away . seriously . Good Old War is acoustic and makes me smile . I really can not help but be happy when I listen to them ; I think it 's the fact that they seemed so happy themselves when they played .","In the future , will this person go to see other bands play ?",None of the above choices .,"This person likes music and likes to see the show , they will see other bands play .","This person only likes Good Old War and Person L , no other bands .",Other Bands is not on tour and this person can not see them .,1


Remove Noise + Clean Dataset

In [ ]:
import re

def clean_punctuation_spacing(row):
    # Remove spaces before and after punctuation marks
    cols = ["context",	"question",	"answer0",	"answer1",	"answer2",	"answer3"]
    for col in cols:
        if len(row[col].strip()) == 0:
            continue
        row[col] = re.sub(r'\s+([,.!?;:])', r'\1', row[col])  # Remove space before punctuation
        row[col] = re.sub(r'([,.!?;:])\s+', r'\1 ', row[col])  # Remove space after punctuation, keep one space
        row[col] = re.sub(r'\s*-\s*', r'-', row[col])   # Remove spaces before and after hyphens (-)
        contractions = ["'s", "'re", "'ve", "'d", "'ll", "n't"]

        for con in contractions:
            row[col] = row[col].replace(' ' + con, con)
    return row

In [ ]:
cosmosqa_dataset = cosmosqa_dataset.apply(clean_punctuation_spacing, axis=1)

In [ ]:
cosmosqa_dataset.head(1)

,context,question,answer0,answer1,answer2,answer3,label
0,"Good Old War and person L: I saw both of these bands Wednesday night, and they both blew me away. seriously. Good Old War is acoustic and makes me smile. I really can not help but be happy when I listen to them; I think it's the fact that they seemed so happy themselves when they played.","In the future, will this person go to see other bands play?",None of the above choices.,"This person likes music and likes to see the show, they will see other bands play.","This person only likes Good Old War and Person L, no other bands.",Other Bands is not on tour and this person can not see them.,1


Extract Useful Data

In [ ]:
final_cosmosqa_dataset = pd.DataFrame(columns=['context', 'question', 'answer', 'dis1', 'dis2', 'dis3', 'source_dataset']) # final cosmosqa dataset

In [ ]:
for r in cosmosqa_dataset.iterrows():
  options = ['answer0', 'answer1', 'answer2', 'answer3']
  if not int(r[1]['label']) in range(0, 4):
    continue
  answer_id = 'answer'+str(r[1]['label']).strip()
  options.remove(answer_id) # options now contain distractors ids

  # get answer and distractors
  answer = r[1][answer_id]
  distractors = [op for op in r[1][options]]
  assert len(distractors) == 3

  dis_lens = [len(str(d).split(' ')) for d in distractors]
  avg_dis_len = pd.Series(dis_lens).mean()

  if avg_dis_len >= AVG_LEN_THRES:
    datapoint = {"context": [r[1]['context']], "question": [r[1]['question']], "answer": [answer], "dis1": [distractors[0]], "dis2": [distractors[1]], "dis3": [distractors[2]], "source_dataset": ['cosmos']}
    final_cosmosqa_dataset = pd.concat([final_cosmosqa_dataset, pd.DataFrame(datapoint)], axis=0, ignore_index=True)

In [ ]:
final_cosmosqa_dataset.shape

(7129, 7)

In [ ]:
final_cosmosqa_dataset.sample(50)

,context,question,answer,dis1,dis2,dis3,source_dataset
2830,"I arrived at the Crown at 1 am bleary eyed and for some reason they confused my surname with Elizabeth and I am convinced they were going to put me in Shannon's room! I woke my wife Ellen up the next morning with a declaration of "" Hey! Where's Shannon Elizabeth and how in hell did you get here! """,Why were you convinced they were going to put you in Shannon's room?,I was convinced they were going to put me in Shannon's room because because they got me and Elizabeth's name mixed up.,I was convinced they were going to put me in Shannon's room because she was clinging onto me while I was bleary eyed.,I was convinced they were going to put me in Shannon's room because they acted as though Elizabeth was my wife.,I was convinced they were going to put me in Shannon's room because they handed me the keys to Elizabeth's roomin.,cosmos
5398,"I have sustained many injuries from the outside world. My house isn't much safer. I run into walls, i trip over toys. I once tripped over one of Brady's toys and slammed into the doors that hide my washer and dryer.",What will happen to the narrator and this house?,The narrator will seek to make the house more safe.,The narrator will seek to make the house sellable.,The narrator will seek to make the house more appealing to buyers.,None of the above choices.,cosmos
3827,"( earlier while said director was inside rotating / opening the dome for photogenicity, someone commented, "" I suppose there's a star up there somewhere. "" I really wanted to say, "" yeah, it's that bright one up there behind you. "" I withheld my snark, aren't you proud?",What is the reason I withheld my snark?,It would have been rude,I would have been arrested,I couldn't think of anything good to say,I wanted to wait until later so I could write it out.,cosmos
6571,"But to the Modern Liberal who can not make that judgment-must not make that judgment-that would be discriminating. They have no explanation. Therefore, the only explanation for success has to be that somehow success has cheated.",What may be the liberal's reason to not make the judgement?,They felt it wasn't their place.,They felt that they didn't want to be in the place.,The liberal was being discriminated against.,They didn't want to cheat their success.,cosmos
2354,"I ' ve known the wolf since the third grade. We ' ve done some pretty fun stuff in our youth, some of which I ' m not sure the statute of limitations have run out on. We weren't criminals per se, but we were no angles either.",How might I feel about The Wolf when I think about him?,I have good memories of him and me doing what young kids do.,I like to remember him as a little criminal who would do anything for a buck.,I've known him since we were in kindergarten and we did everything together.,None of the above choices.,cosmos
1961,"Turns out that they're going to have to perform open heart surgery to fix the problem. They scheduled the surgery for Tuesday the 2nd of September, and they're letting him go right now. My stepmother and my aunt Janet are on their way to go get him right now, and as soon as I get out of work, I ' m going to go stop by his place and see him for a bit. And, I'll be taking Tuesday off to be at the hospital. I hate hospitals. Gahhh!!!",Why is the narrator feeling stressed and anxious?,They have a family member facing an operation.,They have a family member facing a job at the hospital.,They have a family member facing a termination.,They have a family member facing a fight with Janet.,cosmos
6169,Also we have no Internet here either so I had to email via iPhone and each time I pressed delete it deleted a whole sentence which made me cry. Yesterday at the park two dogs ran into me. Hard.,Why is this person so upset?,They has to move where there is no internet.,Two dogs ran into them and made them drop their iphone.,The emailed customer service but didn't get an answer.,None of the above choices.,cosmos
6614,"Ok, I 

Final Dataset

In [ ]:
final_dataset = pd.concat([final_kaggle_dataset, final_race_dataset, final_cosmosqa_dataset], axis=0, ignore_index=True)

In [ ]:
final_dataset = final_dataset.sample(frac=1, ignore_index=True, random_state=42)
final_dataset = final_dataset.sample(frac=1, ignore_index=True, random_state=42)

In [ ]:
final_dataset.shape

(25401, 7)

In [ ]:
def split(data, test_size=0.2):
  test_size = int(len(data) * test_size)
  test_set = data.iloc[:test_size]
  train_set = data.iloc[test_size:]

  return train_set, test_set

In [ ]:
trainset, testset = split(final_dataset)

In [ ]:
print(trainset.shape)
print(testset.shape)

(20321, 7)
(5080, 7)


In [ ]:
trainset.to_csv('./data/trainset.csv', index=False)
testset.to_csv('./data/testset.csv', index=False)

#EDA on Final Dataset

In [ ]:
trainset = pd.read_csv('trainset.csv')
testset = pd.read_csv('testset.csv')
final_dataset = pd.concat([trainset, testset], ignore_index=True)

In [ ]:
ques_lens = [count_words(q) for q in final_dataset['question']]
cont_lens = [count_words(q) for q in final_dataset['context']]

dis_lens = []
for r in final_dataset.iterrows():
  distractors = r[1]['dis1'] + r[1]['dis2'] + r[1]['dis3']
  dis_lens.append(len(distractors.split(' ')))

In [ ]:
plt.title('Question Length')
plt.hist(ques_lens, bins=32, color='green')
plt.show()

plt.title('Context Length')
plt.hist(cont_lens, bins=32, color='purple')
plt.show()

plt.title('Distractor Length')
plt.hist(dis_lens, bins=32)
plt.show()

In [ ]:
# filter

filtered_rows = []
for r in final_dataset.iterrows():
  # if "why" in r[1]['question'] or "because" in r[1]['question']:
  if "_" in r[1]['question'] and r[1]['source_dataset'] == 'race':
    filtered_rows.append(r[0])
len(filtered_rows)

In [ ]:
filtered = final_dataset.iloc[filtered_rows, :]
filtered.shape

In [ ]:
final_dataset[final_dataset['source_dataset'] == 'race'].iloc[50:80, :]